In [90]:
import pandas as pd
import numpy as np
import os
import datetime


In [91]:
## Get address of where the repository is located on your computer.

# Will raise exception is you're in the wrong folder.
if not os.getcwd().endswith("02443-SimSolarEnergy\\code"):
    raise Exception("Change working directory to the code folder.")
repo_loc = os.getcwd()[ : -len("\\code")]


In [92]:
# Load data
activ = pd.read_csv(repo_loc + "\\data\\UK-TUS-15-0616\\activity.csv")
house = pd.read_csv(repo_loc + "\\data\\UK-TUS-15-0616\\household.csv")
indiv = pd.read_csv(repo_loc + "\\data\\UK-TUS-15-0616\\individual.csv")

for data in [activ, house, indiv]:
    print(data.head())
    print("\n\n")

     serial  pnum               t_start  eptime  whatdoing_exact  WhereWhen  \
0  11011202     1  2014-12-11T03:00:00Z     110              110         11   
1  11011202     1  2014-12-11T04:50:00Z      10             8219         11   
2  11011202     1  2014-12-11T05:00:00Z      10              310         11   
3  11011202     1  2014-12-11T05:10:00Z      10             3210         11   
4  11011202     1  2014-12-11T05:20:00Z      10             3110         11   

   whatdoing  
0          0  
1          8  
2          0  
3          3  
4          3  



     serial  num_adult  num_child  num_room  microwav  dishwash  dhhtype
0  11010903          2          0         8      True      True      3.0
1  11010904          2          0         6     False      True      3.0
2  11010906          3          0         4      True     False      7.0
3  11010907          2          1         3      True     False      2.0
4  11010908          1          0         6      True      True    

In [93]:
for data in [indiv]:
    print(data.head())
    print("\n\n")

     serial  pnum  sex  age  deconact
0  11010903     1    1   80       7.0
1  11010903     2    2   71       7.0
2  11010904     1    2   55       2.0
3  11010904     2    1   62       1.0
4  11010906     1    2   52       2.0





In [94]:
# Check for nulls
activ.isna().any()

serial             False
pnum               False
t_start             True
eptime             False
whatdoing_exact    False
WhereWhen          False
whatdoing          False
dtype: bool

In [85]:
# What proportion of time values are null?
len(activ[activ.t_start.isna()])/len(activ)

0.0035107005745092165

In [95]:
# Not many out of the total, so let's just delete them
activ = activ[activ.t_start.isna() == False]

In [96]:
# First of all, only take the 11's
activ = activ[activ.WhereWhen == 11]

# Now change time into time "states"
time_temp = pd.to_datetime(activ.t_start)
activ["time_period"] = (round(time_temp.dt.minute / 10) + time_temp.dt.hour * 6).astype(int)

In [97]:
# Check:
np.sort(activ.time_period.unique())

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143], dtype=int64)

In [87]:
# Looks perfect!
# Change WhatDoing to our prefered (easier) level:
print(activ.whatdoing)
#activ.whatdoing = (round(activ.whatdoing / 10)).astype(int)


0         0
1         8
2         0
3         3
4         3
5         7
6         0
7         3
8         0
9         3
10        3
11        3
12        3
13        3
14        7
15        5
16        5
17        7
18        7
19        3
20        5
21        5
22        8
23        8
24        0
25        0
26        9
27        3
28        3
29        3
         ..
587602    3
587603    3
587604    0
587605    0
587606    8
587607    5
587608    7
587609    8
587610    0
587611    0
587612    0
587613    0
587614    0
587615    3
587616    0
587617    9
587618    3
587619    9
587620    3
587621    6
587622    9
587623    3
587624    0
587625    0
587626    8
587627    9
587628    8
587629    8
587630    0
587631    0
Name: whatdoing, Length: 585569, dtype: int64


In [98]:
# Init
#activ["person_ID"] = activ.serial + activ.pnum # <- Identify each person apart from household
activities = activ.whatdoing.unique() # <- Get all activities
total_tp = 144
print ((activ.head()))
P_t = np.zeros([total_tp, len(activities), len(activities)]) # <- 3D array for 144 * transition matrices    

     serial  pnum               t_start  eptime  whatdoing_exact  WhereWhen  \
0  11011202     1  2014-12-11T03:00:00Z     110              110         11   
1  11011202     1  2014-12-11T04:50:00Z      10             8219         11   
2  11011202     1  2014-12-11T05:00:00Z      10              310         11   
3  11011202     1  2014-12-11T05:10:00Z      10             3210         11   
4  11011202     1  2014-12-11T05:20:00Z      10             3110         11   

   whatdoing  time_period  
0          0           18  
1          8           29  
2          0           30  
3          3           31  
4          3           32  


In [101]:
def keep2(x):
    return int(str(x)[:2])

In [378]:
#joining dataframe indiv and activ over customer serial and pnum
new_df = pd.merge(activ, indiv, how= 'left', left_on = ['serial','pnum'],right_on = ['serial','pnum'])

new_df['whatdoing_exact'] =new_df['whatdoing_exact'].apply(keep2)
new_df = new_df.drop(columns = ['WhereWhen','whatdoing','sex','deconact'])
#new_df.head()

#create two dataframes
df_child = new_df[new_df['age']<18]
df_adult = new_df[new_df['age']>=18]


In [379]:
print(activ.shape)
print(indiv.shape)
print(df_child.shape)
print(df_adult.shape)
print(df_child.head())
print(df_adult.head())

(395400, 8)
(11421, 5)
(41292, 7)
(354108, 7)
      serial  pnum               t_start  eptime  whatdoing_exact  \
79  11011202     4  2014-12-11T03:00:00Z     190               11   
80  11011202     4  2014-12-11T06:10:00Z      10               31   
81  11011202     4  2014-12-11T06:20:00Z      10               21   
82  11011202     4  2014-12-11T06:30:00Z      10               31   
83  11011202     4  2014-12-11T06:40:00Z      10               31   

    time_period  age  
79           18   13  
80           37   13  
81           38   13  
82           39   13  
83           40   13  
     serial  pnum               t_start  eptime  whatdoing_exact  time_period  \
0  11011202     1  2014-12-11T03:00:00Z     110               11           18   
1  11011202     1  2014-12-11T04:50:00Z      10               82           29   
2  11011202     1  2014-12-11T05:00:00Z      10               31           30   
3  11011202     1  2014-12-11T05:10:00Z      10               32           31

In [380]:
#need to convert date time formatting and split it 
def convertandsplitdatetime(df_test): 
    index = df_test.columns.get_loc("t_start")
    df_test['t_start'] = pd.to_datetime(df_test['t_start'], format='%Y-%m-%dT%H:%M:%SZ')
    new_date = [d.date() for d in df_test['t_start']]
    new_time = [d.time() for d in df_test['t_start']]
    #df_test = df_test.drop(columns = ['t_start'])
    df_test.insert (index, "Date", new_date)
    df_test.insert (index+1, "Time", new_time)
    return df_test

# need to duplicate rows and chenge their eptime
def getmultitimesteps_fixedlenght(df_test): 
    df_test.eptime = df_test.eptime.astype(float)
    df_test = df_test.loc[df_test.index.repeat((df_test.eptime/10).astype(int))]
    df_test.loc[df_test['eptime'] > 10, 'eptime' ] = 10
    #df_test.head(15)
    #df_test.shape
    return df_test

# add ten minutes to time 
def adjust_timestamps(setoftimes):
    #if len(setoftimes)>1:
    if (isinstance(setoftimes, datetime.time) == False):
        counter = 0
        firsttime = setoftimes.head(1).item()
        newSetOfTimes = []
        for element in setoftimes:
            deltaminutes = dt.timedelta(minutes =counter*10)
            whatisthis = dt.datetime.combine(dt.date(1,1,1),element)
            element = (dt.datetime.combine(dt.date(1,1,1),element) + deltaminutes).time()
            print (counter, element,deltaminutes,whatisthis)
            counter +=1
            newSetOfTimes.append(element)
        newSetOfTimes = pd.Series(newSetOfTimes)
    else: 
        newSetOfTimes = setoftimes
    return newSetOfTimes

#updates all the groups of times that have the same index (adding 10mins progressively)
def updatealltimes(df_test):
    #df_test.assign('New Time'="")
    df_test['New Time'] = ""
    allindx = df_test.index.values.tolist()
    for i in allindx:
        setoftimes= df_test['Time'][i]
        check = type(setoftimes)
        if (isinstance(setoftimes, datetime.time) == False):
            newSetOfTimes = adjust_timestamps(setoftimes)
            #df_test['New Time'][i]= newSetOfTimes
            df_test['New Time'].append(newSetOfTimes)
        else:
            #df_test['New Time'][i]= setoftimes
            df_test['New Time'].append(setoftimes)
        if i % 10000 == 0:
            print (i)
    return df_test
    

In [381]:
# this returns some red stuf... but it works! 
df_child =  convertandsplitdatetime(df_child)
df_adult =  convertandsplitdatetime(df_adult)

C:\Users\arita\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [382]:
print(df_child.head())
print(df_adult.head())

      serial  pnum        Date      Time             t_start  eptime  \
79  11011202     4  2014-12-11  03:00:00 2014-12-11 03:00:00     190   
80  11011202     4  2014-12-11  06:10:00 2014-12-11 06:10:00      10   
81  11011202     4  2014-12-11  06:20:00 2014-12-11 06:20:00      10   
82  11011202     4  2014-12-11  06:30:00 2014-12-11 06:30:00      10   
83  11011202     4  2014-12-11  06:40:00 2014-12-11 06:40:00      10   

    whatdoing_exact  time_period  age  
79               11           18   13  
80               31           37   13  
81               21           38   13  
82               31           39   13  
83               31           40   13  
     serial  pnum        Date      Time             t_start  eptime  \
0  11011202     1  2014-12-11  03:00:00 2014-12-11 03:00:00     110   
1  11011202     1  2014-12-11  04:50:00 2014-12-11 04:50:00      10   
2  11011202     1  2014-12-11  05:00:00 2014-12-11 05:00:00      10   
3  11011202     1  2014-12-11  05:10:00 201

In [383]:
df_adult  = df_adult.drop(columns = ['t_start'])
df_child = df_child.drop(columns = ['t_start'])

In [384]:
# do this as last... will increase a lot the number of rows
# will also add rows with THE SAME INDEX AND TIME STEP
# we fix it later
df_adult = getmultitimesteps_fixedlenght(df_adult)
df_child = getmultitimesteps_fixedlenght(df_child)

In [385]:
print(df_child.shape)
print(df_adult.shape)

(217790, 8)
(1446536, 8)


In [386]:
df_child.head()
# we have split the 190 duration in 19 lines of duration 10...
# but they have the same index nd time
# we fix that with function updatealltimes

,serial,pnum,Date,Time,eptime,whatdoing_exact,time_period,age
79,11011202,4,2014-12-11,03:00:00,10.0,11,18,13
79,11011202,4,2014-12-11,03:00:00,10.0,11,18,13
79,11011202,4,2014-12-11,03:00:00,10.0,11,18,13
79,11011202,4,2014-12-11,03:00:00,10.0,11,18,13
79,11011202,4,2014-12-11,03:00:00,10.0,11,18,13


In [387]:
import datetime
df_test = df_child.head(100)
df_test= updatealltimes(df_test)

0 03:00:00 0:00:00 0001-01-01 03:00:00
1 03:10:00 0:10:00 0001-01-01 03:00:00
2 03:20:00 0:20:00 0001-01-01 03:00:00
3 03:30:00 0:30:00 0001-01-01 03:00:00
4 03:40:00 0:40:00 0001-01-01 03:00:00
5 03:50:00 0:50:00 0001-01-01 03:00:00
6 04:00:00 1:00:00 0001-01-01 03:00:00
7 04:10:00 1:10:00 0001-01-01 03:00:00
8 04:20:00 1:20:00 0001-01-01 03:00:00
9 04:30:00 1:30:00 0001-01-01 03:00:00
10 04:40:00 1:40:00 0001-01-01 03:00:00
11 04:50:00 1:50:00 0001-01-01 03:00:00
12 05:00:00 2:00:00 0001-01-01 03:00:00
13 05:10:00 2:10:00 0001-01-01 03:00:00
14 05:20:00 2:20:00 0001-01-01 03:00:00
15 05:30:00 2:30:00 0001-01-01 03:00:00
16 05:40:00 2:40:00 0001-01-01 03:00:00
17 05:50:00 2:50:00 0001-01-01 03:00:00
18 06:00:00 3:00:00 0001-01-01 03:00:00
0 03:00:00 0:00:00 0001-01-01 03:00:00
1 03:10:00 0:10:00 0001-01-01 03:00:00
2 03:20:00 0:20:00 0001-01-01 03:00:00
3 03:30:00 0:30:00 0001-01-01 03:00:00
4 03:40:00 0:40:00 0001-01-01 03:00:00
5 03:50:00 0:50:00 0001-01-01 03:00:00
6 04:00:00 1:00:

C:\Users\arita\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


TypeError: cannot concatenate object of type "<class 'datetime.time'>"; only pd.Series, pd.DataFrame, and pd.Panel (deprecated) objs are valid

In [389]:
df_test.head(100)

,serial,pnum,Date,Time,eptime,whatdoing_exact,time_period,age,New Time
79,11011202,4,2014-12-11,03:00:00,10.0,11,18,13,
79,11011202,4,2014-12-11,03:00:00,10.0,11,18,13,
79,11011202,4,2014-12-11,03:00:00,10.0,11,18,13,
79,11011202,4,2014-12-11,03:00:00,10.0,11,18,13,
79,11011202,4,2014-12-11,03:00:00,10.0,11,18,13,
79,11011202,4,2014-12-11,03:00:00,10.0,11,18,13,
79,11011202,4,2014-12-11,03:00:00,10.0,11,18,13,
79,11011202,4,2014-12-11,03:00:00,10.0,11,18,13,
79,11011202,4,2014-12-11,03:00:00,10.0,11,18,13,
79,11011202,4,2014-12-11,03:00:00,10.0,11,18,13,


In [312]:
#from datetime import datetime  
import datetime as dt
from datetime import timedelta 

print (type(setoftimes))
dfcindx = df_child.index.values.tolist()
for i in dfcindx:
    setoftimes= df_child['Time'][i] # this takes the all group of wrongly indexed df
    print (type(setoftimes))
    if len(setoftimes)>1:
    #if sum(setoftimes.value_counts) > 1:
        counter = 0
        firsttime = setoftimes.head(1).item()
        newSetOfTimes = []
        for element in setoftimes:
            deltaminutes = dt.timedelta(minutes =counter*10)
            element = (dt.datetime.combine(dt.date(1,1,1),element) + deltaminutes).time()
            counter +=1
            newSetOfTimes.append(element)
            print (element)

    newSetOfTimes = pd.Series(newSetOfTimes)      
    #print (type(newSetOfTimes))
    print (newSetOfTimes)

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
03:00:00
15:50:00
04:40:00
17:30:00
06:20:00
19:10:00
08:00:00
20:50:00
09:40:00
22:30:00
11:20:00
00:10:00
13:00:00
01:50:00
14:40:00
03:30:00
16:20:00
05:10:00
18:00:00
0     03:00:00
1     15:50:00
2     04:40:00
3     17:30:00
4     06:20:00
5     19:10:00
6     08:00:00
7     20:50:00
8     09:40:00
9     22:30:00
10    11:20:00
11    00:10:00
12    13:00:00
13    01:50:00
14    14:40:00
15    03:30:00
16    16:20:00
17    05:10:00
18    18:00:00
dtype: object
<class 'pandas.core.series.Series'>
03:00:00
15:50:00
04:40:00
17:30:00
06:20:00
19:10:00
08:00:00
20:50:00
09:40:00
22:30:00
11:20:00
00:10:00
13:00:00
01:50:00
14:40:00
03:30:00
16:20:00
05:10:00
18:00:00
0     03:00:00
1     15:50:00
2     04:40:00
3     17:30:00
4     06:20:00
5     19:10:00
6     08:00:00
7     20:50:00
8     09:40:00
9     22:30:00
10    11:20:00
11    00:10:00
12    13:00:00
13    01:50:00
14    14:40:00
15    03:30:00
16    16:20

TypeError: object of type 'datetime.time' has no len()

In [215]:
#finally resetting the indexes
df_adult  = df_adult.reset_index()
df_child = df_child.reset_index() 

In [227]:
df_test  = df_child
#df_test.t_start = df_test.t_start.astype(float)
#df_test  = df_test.reset_index() 

#df_test['t_start'] = pd.to_datetime(df_test['t_start'], format='%Y-%m-%dT%H:%M:%SZ')
#df_test['new_date'] = [d.date() for d in df_test['t_start']]
#df_test['new_time'] = [d.time() for d in df_test['t_start']]
#df_test.loc[df_test['t_start'] > 10, 't_start' ] = 10
df_test.head(210)
#df_test.shape

,serial,pnum,Date,Time,eptime,whatdoing_exact,time_period,age
79,11011202,4,2014-12-11,03:00:00,10.0,11,18,13
79,11011202,4,2014-12-11,03:00:00,10.0,11,18,13
79,11011202,4,2014-12-11,03:00:00,10.0,11,18,13
79,11011202,4,2014-12-11,03:00:00,10.0,11,18,13
79,11011202,4,2014-12-11,03:00:00,10.0,11,18,13
79,11011202,4,2014-12-11,03:00:00,10.0,11,18,13
79,11011202,4,2014-12-11,03:00:00,10.0,11,18,13
79,11011202,4,2014-12-11,03:00:00,10.0,11,18,13
79,11011202,4,2014-12-11,03:00:00,10.0,11,18,13
79,11011202,4,2014-12-11,03:00:00,10.0,11,18,13


In [ ]:
#swipe df_adult 
for i in range(len(df_adult)):
    


# Loop through time periods
for tp in range(total_tp):
    data_in_tp = activ[activ.time_period == tp]
    activ_in_tp[i] = len(data_in_tp)
    
    # Loop through individuals
    for person in data_in_tp.person_ID.unique():
        pass